In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as LA
import math
import matplotlib.pyplot as plt

def Adjacency(C):
    Net =[[ [], -1.0 ]]
    Net2 =[[ [], -1.0 ]]
    time_points = C[:,2]
    nodes0 = C[:,0]
    nodes1 = C[:,1]
    num1 = int(max(np.unique(nodes0)))
    num2 = int(max(np.unique(nodes1)))
    num = max([num1,num2])
    N = np.shape(C[:,0])[0]
    time_points = np.unique(C[:,2])
    
    for t in time_points:
        A = np.zeros((num+1,num+1))
        for k in range(N):
            if C[k][2] == t:
                n = int(C[k][0])
                m = int(C[k][1])
                A[n][m] = 1
                A[m][n] = 1
        A2 = A.tolist()
        Net.append([A2,t])
    return Net

def ComputeBk(net_k, t_k, alpha, k ,B_k_1,t_k_1):
    if k == 1 :
        net1  = net_k
        t1 = t_k
        B1 = np.copy(net1)
        return B1, t1

    numrow = len(net_k)
    numcol = len(net_k[0])
    Bk = np.zeros((numrow, numcol))
    for i in range(0, numrow):
        for j in range(0,numcol):
            node_k   = net_k[i][j]
            if node_k != 0:
                update = node_k + ( B_k_1[i][j]/math.exp( alpha*(t_k-t_k_1) ) )
            else:
                update = 0 + ( B_k_1[i][j]/math.exp( alpha*(t_k-t_k_1) ) )

            Bk[i][j] = update
    return Bk,t_k

def Tie_decay_matrices(alpha, C):
    Net = Adjacency(C)
    T = []
    B = [ [ [], -1.0 ] ]
    for i, n_i in enumerate(Net):
        if i==0 :
            continue
        Bi_1   = B[i-1]
        Bi,Ti  = ComputeBk(n_i[0],n_i[1] ,alpha, i, Bi_1[0],Bi_1[1])
        B.append([Bi,Ti])
    for j in range(1,len(B)):
        T.append(B[j][0])
    return T,B

def Find_eigenpair(D):
    A = []
    eigenvalues = LA.eig(D)[0]
    eigenvectors_col = LA.eig(D)[1]
    
    idx = eigenvalues.argsort()[::-1]   
    eigenvalues = eigenvalues[idx]
    eigenvectors_col = eigenvectors_col[:,idx]
    
    eigenvectors_row = np.transpose(eigenvectors_col)
    for i in range(len(eigenvalues)):
        A.append([eigenvalues[i],eigenvectors_row[i]])
    return A

def Centered_Distance_matrix(D):
    n = np.shape(D)[0]
    m = np.shape(D)[1]
    I = np.identity(n)
    One_mat = np.ones((n,n))
    H = I - (1/n * One_mat)
    D_scaled = -0.5 * np.matmul(H,np.matmul(D,H))
    return D_scaled

def Squared_Frobenius_Distance_matrix(T): #T is from tie_decay_matrices(alpha,C)
    D3 = np.zeros((len(T),len(T)))
    for i, ti in enumerate(T):
        for j, tj in enumerate(T):
            if i == j:
                D3[i][j] = 0
                D3[j][i] = 0
            elif i < j:
                D3[i][j] = LA.norm(ti-tj)**2
                D3[j][i] = D3[i][j]
            else:
                continue
    return D3

def Degree_matrix(A): #A is a tie-decay adjacency matrix at a time t.
    m = np.shape(A)[0]
    n = np.shape(A)[1]
    degree_matrix = np.zeros((m,n))
    
    for i in range(0,m):
        r = sum(A[i])
        degree_matrix[i][i] = degree_matrix[i][i] + r

    return degree_matrix

def Laplacian_matrix(A): #A is also a tie-decay adjacency matrix at a time t.
    L = Degree_matrix(A) - A
    
    return L

def Laplacian_distance(A,B): # A,B are tie-decay matrices. Already squared.
    L = Laplacian_matrix(A)
    K = Laplacian_matrix(B)
    M, N = Find_eigenpair(L), Find_eigenpair(K)
    m = len(M)
    #n = len(N)
    value = 0
    
    for i in range(0,m):
        value = value + ((np.real(M[i][0]) - np.real(N[i][0]))**2)
        
    #value = np.sqrt(value)
    return value

def Laplacian_distance_matrix(T): # T = Tie_decay_matrices(alpha, C)
    LD = np.zeros((len(T),len(T)))
    
    for i, ti in enumerate(T):
        for j, tj in enumerate(T):
            if i == j:
                LD[i][j] = 0
                LD[j][i] = 0
            elif i < j:
                LD[i][j] = Laplacian_distance(ti,tj)
                LD[j][i] = LD[i][j]
            else:
                continue
            
    return LD

def Labeled_distance(A,B): # A,B are tie-decay matrices.
    L = Laplacian_matrix(A)
    K = Laplacian_matrix(B)
    M, N = Find_eigenpair(L), Find_eigenpair(K)
    m = len(M)
    n = len(N)
    value = 0
    for i in range(0,m):
        for j in range(0,n):
            if np.round(M[i][0] + N[j][0], 20) != 0:
                part1 = M[i][0] - N[j][0]
                part1 = (part1)**2
                part2 = M[i][0] + N[j][0]
                part2 = 1000000*part2
                part3 = np.dot(M[i][1],N[j][1])
                part3 = (part3)**2
                numerator = 1000000 * part1 * part3
                value = value + (numerator)/part2
            #value = value + ((1000*((M[i][0] - N[j][0])**2) / 1000*(M[i][0] + N[j][0])) * (np.dot(M[i][1],N[j][1])**2))
                value = np.real(value)
            else:
                value = value + 0
    return value

def Labeled_distance_matrix(T): # T = Tie_decay_matrices(alpha, C)
    Lab_D = np.zeros((len(T),len(T)))
    
    for i, ti in enumerate(T):
        for j, tj in enumerate(T):
            if i == j:
                Lab_D[i][j] = 0
            else:
                Lab_D[i][j] = Labeled_distance(ti,tj)**2
            
    return Lab_D #Already squared

def aux_delta(B_ast):
    B = B_ast
    N = len(B)
    P = []
    X = []
    T = []
    B_new = []
    for j in range(1, N):
        T.append(B[j][1]) #Now T's index starts at 0.
        B_new.append(B[j][0]) #Now B_new's index begins from 0.
        
    for ti in B_new:
        L = Laplacian_matrix(ti)
        M = Find_eigenpair(L)
        m = len(M)
        Q = []
        Y = []
        for k in range(0,m):
            Q.append(M[k][0])
            Y.append(M[k][1])
        P.append(Q)
        X.append(Y)
    
    return B_new,T,P,X
    
def delta(t,alpha,aux_delta,type_d): #C is the matrix of Raw data.
    B_new = aux_delta[0]
    T = aux_delta[1]
    P = aux_delta[2]
    X = aux_delta[3]
    deltaa = []
        
    for i in range(0,len(T)-1):
        if t >= T[i] and t < T[i+1]:
            B_t = B_new[i]# / math.exp(alpha *(t-T[i]))
            num_i = i
            break
        if t >= T[len(B_new)-1]:
            B_t = B_new[len(B_new)-1]# / math.exp(alpha *(t-T[len(B_new)-1]))
            num_i = len(B_new)-1
            break

    if type_d == 1:
        for k in range(0,len(B_new)):
            c = np.exp(alpha *(T[num_i]-t))
            c1 = c**2 - c
            part1 = c1 * (LA.norm(B_t)**2)
            c2 = 1 - c
            part2 = c2 * (LA.norm(B_new[k])**2)
            part3 = c*(LA.norm(B_t-B_new[k])**2)
            value = part1 + part2 + part3
            deltaa.append(value)
            #value = np.sqrt(value)
            #deltaa.append(value)
        return deltaa, T
   
    elif type_d == 2:
        U = []
        Q = []
        L = Laplacian_matrix(B_t)
        M = Find_eigenpair(L)
        m = len(M)
        for k in range(0,m):
            U.append(M[k][0])
            Q.append(M[k][1])
        c = np.exp(alpha *(T[num_i]-t))
        for k in range(0,len(B_new)):
            value = 0
            for l in range(0,len(U)):
                part1 = c*U[l]
                part2 = part1 - P[k][l]
                value = value + (part2)**2
            #value = np.sqrt(value)
            deltaa.append(value)
        return deltaa, T

    elif type_d == 3:
        U = []
        Q = []
        L = Laplacian_matrix(B_t)
        M = Find_eigenpair(L)
        m = len(M)
        for k in range(0,m):
            U.append(M[k][0])
            Q.append(M[k][1])
        c = np.exp(alpha *(T[num_i]-t))
        
        for k in range(0,len(B_new)):
            V = P[k]
            R = X[k]
            value = 0
            for l in range(0,len(U)):
                for h in range(0,len(V)):
                    if np.round(U[l]+V[h],10) != 0:
                        p1 = (c * U[l]) - V[h]
                        p1 = (p1)**2
                        p2 = (c * U[l]) + V[h]
                        p2 = 1000000 * p2
                        p3 = np.dot(Q[l],R[h])
                        p3 = (p3)**2
                        numer = 1000000 * p1 * p3
                        value = value + (numer / p2)
                        value = np.real(value)
                    else:
                        value = value + 0
            deltaa.append(value**2)
        return deltaa, T


# def delta(t,alpha,C,B_ast,type_d): #C is the matrix of Raw data. And B_ast = Tie_decay_matrices(alpha,C)[1]
#     B = B_ast
#     N = len(B)
#     T = []
#     B_new =[]
#     deltaa = []
    
#     for j in range(1, N):
#         T.append(B[j][1]) #Now T's index starts at 0.
#         B_new.append(B[j][0]) #Now B_new's index begins from 0.
        
#     for i in range(0,len(T)-1):
#         if t >= T[i] and t < T[i+1]:
#             B_t = B_new[i] / math.exp(alpha *(t-T[i]))
#             break
#         if t >= T[len(B_new)-1]:
#             B_t = B_new[len(B_new)-1] / math.exp(alpha *(t-T[len(B_new)-1]))
#             break
    
#     if type_d == 1:
#         for k in range(0,len(B_new)):
#             deltaa.append(LA.norm(B_t-B_new[k])**2)
#         return deltaa, T
    
#     elif type_d == 2:
#         for k in range(0,len(B_new)):
#             deltaa.append(Laplacian_distance(B_t,B_new[k]))
#         return deltaa, T
#     elif type_d == 3:
#         for k in range(0,len(B_new)):
#             deltaa.append(Labeled_distance(B_t,B_new[k])**2)
#         return deltaa, T    

def Col_sum(D):
    N = np.shape(D)[1]
    Col_D_sum = np.reshape(np.sum(D, axis = 1), (N,1))
    Delta_mu = 1/N * Col_D_sum
    
    return Delta_mu

def Classical_MDS(D,k): # D is the squared distance
    L = []
    #D = normalisation(D)
    B = Centered_Distance_matrix(D)
    #B = 1/np.std(B_ast) * B_ast
    Lambda = Find_eigenpair(B) # Recall that the eigenvalues are already sorted.
    total_sum = 0
    neg_sum = 0
    confidence = 0
    sqrt_eigenvalue = []
    for i in range(0,len(Lambda)):
        if Lambda[i][0] > 0:
            sqrt_eigenvalue.append(np.sqrt(Lambda[i][0]))

    for i in range(0,k):
        L.append(sqrt_eigenvalue[i] * Lambda[i][1])

    return L, Lambda

def Landmark_MDS(D,Eig_MDS,Col,k,t,alpha,aux_delta,type_d):# Eig_MDS = Classical_MDS(D,k)[1]
    Landmark_eigenpair = Eig_MDS #These eigenpairs already dependends on the choice of distance measure.
    N = np.shape(D)[1]
    Delta_mu = Col
    inv_sqrt_eigenvalues = []
    L_prime_before = []
    for i in range(0,k):
        if Landmark_eigenpair[i][0] > 0:
            inv_sqrt_eigenvalues.append(1 / np.sqrt(Landmark_eigenpair[i][0]))
    
    for i in range(0,k):
        vi = inv_sqrt_eigenvalues[i] * Landmark_eigenpair[i][1]
        L_prime_before = np.concatenate((L_prime_before,vi.tolist()), axis = 0)
        
    L_prime = np.reshape(L_prime_before, (k,N))
    
    Delta_B = delta(t,alpha,aux_delta,type_d)[0]
    Delta_B = np.array(Delta_B)
    Delta_B = np.reshape(Delta_B, (N,1))
    Del = Delta_B - Delta_mu
    return -0.5 * np.matmul(L_prime,Del)

def Information_Loss(Lambda,k): # Recall that the eigenvalues are already sorted.
    total_sum = 0
    neg_sum = 0
    confidence = 0
    sqrt_eigenvalue = []
    for i in range(0,len(Lambda)):
        if Lambda[i][0] > 0:
            print('The ',i+1,'-th eigenvalue is ', Lambda[i][0])

        else:
            neg_sum = neg_sum + abs(Lambda[i][0])
            print('Negative/Zero eigenvalues:','number',i+1,'with value',Lambda[i][0])
    for j in range(0,len(Lambda)):
         total_sum = total_sum + abs(Lambda[j][0])
    
    for a in range(0,k):
        print('The ',a+1,'-th eigenvalue is responsible for ', (Lambda[a][0]/total_sum) * 100, '%')
        confidence = confidence + (Lambda[a][0]/total_sum) * 100
    print('Chosen positive eigenvalues ratio: ', confidence, '%')
    print('Negative eigenvalues ratio: ', (neg_sum / total_sum)*100, '%')
    print('The loss information from unchosen +eigen and disrgarding -eigen: ', 100 - confidence, '%')